In [15]:
import requests

from bs4 import BeautifulSoup

from scholarly import scholarly

import re

import pandas as pd

import time

import numpy as np

following the guidance of: https://medium.com/@nandinisaini021/scraping-publications-of-aerial-image-research-papers-on-google-scholar-using-python-a0dee9744728
LH: fix the issue of getting citations
TODO: getting abstract. It seems impossible to get it from google scholar??


In [ ]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}


In [4]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):

  #download the page
  response=requests.get(url,headers=headers)

  # check successful response
  if response.status_code != 200:
    print('Status code:', response.status_code)
    raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
  paper_doc = BeautifulSoup(response.text,'html.parser')

  return paper_doc

In [5]:
def get_tags(doc):
  # select finds multiple instances and returns a list, find finds the first
  paper_tag = doc.select('[data-lid]')
  #cite_tag = doc.select('[title=Cite] + a')
  cite_tag0 = doc.find_all("a",class_='gs_or_cit gs_or_btn gs_nph')
  cite_tag = cite_tag0.copy()
  for i in range(len(cite_tag0)):
    full_text = cite_tag0[i]
    this_two = full_text.find_next("a")
    cite_tag[i]= this_two
  

  link_tag = doc.find_all('h3',{"class" : "gs_rt"})
  author_tag = doc.find_all("div", {"class": "gs_a"})

  return paper_tag,cite_tag,link_tag,author_tag

In [6]:
# it will return the title of the paper
def get_papertitle(paper_tag):
  
  paper_names = []
  
  for tag in paper_tag:
    paper_names.append(tag.select('h3')[0].get_text())

  return paper_names

In [7]:
# it will return the number of citation of the paper
def get_citecount(cite_tag):
  cite_count = []
  for i in cite_tag:
    cite = i.text
    if i is None or cite is None:  # if paper has no citatation then consider 0
      cite_count.append(0)
    else:
      tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
      if tmp is None :
        cite_count.append(0)
      else :
        cite_count.append(int(tmp.group()))

  return cite_count

In [8]:
# function for the getting link information
def get_link(link_tag):

  links = []

  for i in range(len(link_tag)) :
    links.append(link_tag[i].a['href']) 

  return links 

In [16]:
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
  years = []
  publication = []
  authors = []
  for i in range(len(authors_tag)):
      authortag_text = (authors_tag[i].text).split()
      try: #LH
        year = int(re.search(r'\d+', authors_tag[i].text).group())
      except:
        year = np.nan
      years.append(year)
      publication.append(authortag_text[-1])
      author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
      authors.append(author)
  
  return years , publication, authors

In [10]:
# adding information in repository
def add_in_paper_repo(papername,year,author,cite,publi,link):
  paper_repos_dict['Paper Title'].extend(papername)
  paper_repos_dict['Year'].extend(year)
  paper_repos_dict['Author'].extend(author)
  paper_repos_dict['Citation'].extend(cite)
  paper_repos_dict['Publication'].extend(publi)
  paper_repos_dict['Url of paper'].extend(link)

  return pd.DataFrame(paper_repos_dict)

In [19]:
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }
for i in range (0,490,10):
  print(i)


  # get url for the each page
  # open google scholar, search with keywords, click: next page, copy and paste the url below
  # delete the numbers after start=, and add .format(i) at the end.
  #url = 'https://scholar.google.com/scholar?start={}&q=Burkholderia+Pseudomallei&hl=en&as_sdt=0,5'.format(i)
  url = 'https://scholar.google.com/scholar?start={}&q=Coastal+Surge,+model,++texas&hl=en&as_sdt=0,44'.format(i)
  # function for the get content of each page
  doc = get_paperinfo(url)

  # function for the collecting tags
  paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)

  # paper title from each page
  papername = get_papertitle(paper_tag)
  
  # year , author , publication of the paper
  year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
  cite = get_citecount(cite_tag)

  # url of the paper
  link = get_link(link_tag)

  # add in paper repo dict
  final = add_in_paper_repo(papername,year,author,cite,publication,link)
  
  # use sleep to avoid status code 429
  time.sleep(30)




0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
Status code: 429


Exception: Failed to fetch web page 

In [ ]:
# save to your path
final.to_csv('/Users/vivianhuang/Documents/GitHub/google-scholar-search/csv_results/coastal_surge_model_texas_500.csv', sep=',', index=False,header=True)